# Homework: Search Evaluation

In [12]:
!pip uninstall minsearch -y
!pip install -U minsearch qdrant_client


  Using cached minsearch-0.0.4-py3-none-any.whl.metadata (8.1 kB)
Using cached minsearch-0.0.4-py3-none-any.whl (11 kB)


### Evaluation data

In [13]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

Here, `documents` contains the documents from the FAQ database with unique IDs, and `ground_truth` contains generated question-answer pairs.

### we will need the following code for evaluating retrieval: 

In [37]:
from tqdm.auto import tqdm

# Measures if at least one relevant document is found in the top k results.
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

#  Average rank of the first relevant document across queries.
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

### Q1. Minsearch text

Now let's evaluate our usual minsearch approach, but tweak the parameters. Let's use the following boosting params:

```python
 boost = {'question': 1.5, 'section': 0.1} 
```

In [33]:
from minsearch import Index

boost = {'question': 1.5, 'section': 0.1}

# initalize our index
index = Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=[]
)
index.fit(documents) # making out document indexable 


In [34]:
print(f"Total queries: {len(ground_truth)}")
print("Example query:", ground_truth[0])



Total queries: 4627
Example query: {'question': 'When does the course begin?', 'course': 'data-engineering-zoomcamp', 'document': 'c02e79ef'}


In [35]:
# search function for a certain query

def search_function(q):
    return index.search(
        query=q['question'],
        filter_dict=None,
        boost_dict=boost,
        num_results=10
    )

### Now we will feed each question from `ground_truth` to our `search_function` (minsearch), then we will compare the result from the search to the ground_truth answers.

In [36]:
metrics = evaluate(ground_truth, search_function)
print(metrics)


  0%|          | 0/4627 [00:00<?, ?it/s]

{'hit_rate': 0.8597363302355738, 'mrr': 0.6897542375497872}


# `Q1-Answer -> 0.85 and the closer answer is 0.84` 